<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/2-1.%20D%EB%B6%84%EB%A5%98%20%EC%A6%9D%EB%B6%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화  'Colab Notebooks'  'My Drive'	'실습용 자료.zip의 사본'


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

In [5]:
ts=pd.read_csv('/content/gdrive/My Drive/답안 작성용 파일.csv' , encoding = "euc-kr")

FileNotFoundError: ignored

## 데이터 전처리

In [6]:
df=df[(df['digit_1'] =='D')] #조건에 맞는 데이터 출력
#ts=ts[(ts['digit_1'] =='D')] #조건에 맞는 데이터 출력

In [7]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0001519,D,35,351,태양광,전기를 모아,태양력발전
1,id_0002425,D,35,351,태양광,전기생산,태양광발전소
2,id_0002666,D,35,351,태양광,전기생산,태양력발전업
3,id_0002762,D,35,351,태양광,전기생산,판매
4,id_0003695,D,35,351,태양광,전기생산,한전에판매
...,...,...,...,...,...,...,...
751,id_0992466,D,35,351,태양광이용,태양광발전,전기생산
752,id_0993939,D,35,351,태양광이용,태양광발전,태양광전기생산
753,id_0994334,D,35,351,태양광이용,탱양광발전,전기생산
754,id_0995674,D,35,352,사업장에서,도시가스를 배관시설에 의해,"가정,산업에 공급"


In [8]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0001519,D,35,351,태양광 전기를 모아 태양력발전
1,id_0002425,D,35,351,태양광 전기생산 태양광발전소
2,id_0002666,D,35,351,태양광 전기생산 태양력발전업
3,id_0002762,D,35,351,태양광 전기생산 판매
4,id_0003695,D,35,351,태양광 전기생산 한전에판매
...,...,...,...,...,...
751,id_0992466,D,35,351,태양광이용 태양광발전 전기생산
752,id_0993939,D,35,351,태양광이용 태양광발전 태양광전기생산
753,id_0994334,D,35,351,태양광이용 탱양광발전 전기생산
754,id_0995674,D,35,352,"사업장에서 도시가스를 배관시설에 의해 가정,산업에 공급"


In [9]:
df['digit_3'].value_counts()

351    660
352     67
353     29
Name: digit_3, dtype: int64

## **# 데이터 증분 및 맞춤법 교정**

In [10]:
# 맞춤법 검사

In [11]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-6m33o04k
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-6m33o04k
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=1e4ac30a19cce8752a5d2571ab42cd76ca4dbbad56275397ecfd73c8ec3bec4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-sik43i6s/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [12]:
from hanspell import spell_checker
temp_result=[]
for sentence in df['text']:
  result=spell_checker.check(sentence)
  temp_result.append(result.checked)

df['text']=pd.DataFrame(temp_result)  

In [13]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [14]:
df1=df[df['digit_3']==352] #8개 증분 필요
df2=df[df['digit_3']==353] #21개 증분 필요
df3=df[df['digit_3']==351]

In [15]:
df1=df1.reset_index(drop=True)

In [16]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df1['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=8)
  id_list=['p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]),'p'+str(df1['AI_id'][n]),'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]),'p'+str(df1['AI_id'][n]),'p'+str(df1['AI_id'][n])]
  digit_1_list=[df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n]]
  digit_2_list=[df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n]]
  digit_3_list=[df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df1_to_insert = pd.DataFrame(data_to_insert)
  df1= df1.append(df1_to_insert, ignore_index=True)
  n=n+1

In [17]:
df1 #df1 완성함

,AI_id,digit_1,digit_2,digit_3,text
0,id_0009566,D,35,352,사업장에서 음식물 쓰레기와 폐수 혐기성 소화를 이용하여 가스 생산
1,id_0010075,D,35,352,가스 배관 공급업 사업체 및 가정에 가스 공급 도시가스
2,id_0018122,D,35,352,가스 배관 라인을 통해 가스 제조 및 배관 공급업
3,id_0027650,D,35,352,LNG(가스) 가스압력 조적(안전하게) 소비자에게 도시가스 공급
4,id_0057362,D,35,352,"가스 분해, 분리, 화락 처리 가스 공급"
...,...,...,...,...,...
665,pid_0997719,D,35,352,사업장에서 도시가스배관 도시가스
666,pid_0997719,D,35,352,사업장에서 공급
667,pid_0997719,D,35,352,공급 도시가스배관 도시가스 사업장에서 공급
668,pid_0997719,D,35,352,사업장에서 공급 도시가스 공급 도시가스배관


In [21]:
df2=df[df['digit_3']==353] #21개 증분 필요

In [25]:
df2=df2.reset_index(drop=True)
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df2['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=20)
  id_list=['p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]), 'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n]),'p'+str(df2['AI_id'][n])]
  digit_1_list=[df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n], df2['digit_1'][n]]
  digit_2_list=[df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n], df2['digit_2'][n]]
  digit_3_list=[df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n], df2['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df2_to_insert = pd.DataFrame(data_to_insert)
  df2= df2.append(df2_to_insert, ignore_index=True)
  n=n+1

In [26]:
df2['digit_3'].value_counts()

353    638
Name: digit_3, dtype: int64

기존 데이터 합치기

In [27]:
df=pd.concat([df1, df2, df3])

In [29]:
df['digit_3'].value_counts()

352    670
351    660
353    638
Name: digit_3, dtype: int64

In [30]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0009566,D,35,352,사업장에서 음식물 쓰레기와 폐수 혐기성 소화를 이용하여 가스 생산
1,id_0010075,D,35,352,가스 배관 공급업 사업체 및 가정에 가스 공급 도시가스
2,id_0018122,D,35,352,가스 배관 라인을 통해 가스 제조 및 배관 공급업
3,id_0027650,D,35,352,LNG(가스) 가스압력 조적(안전하게) 소비자에게 도시가스 공급
4,id_0057362,D,35,352,"가스 분해, 분리, 화락 처리 가스 공급"
...,...,...,...,...,...
749,id_0989325,D,35,351,태양광을 이용하여 태양열 전기 생산
750,id_0989511,D,35,351,사업장에서 의뢰에 의해 전기 공급
751,id_0992466,D,35,351,태양광 이용 태양광발전 전기 생산
752,id_0993939,D,35,351,태양광 이용 태양광발전 태양광 전기 생산


In [31]:
df.to_csv('df_D_inc.csv', index=False, encoding='utf-8-sig')